In [1]:
import numpy as np
import pandas as pd
import pickle

In [21]:
# read data
data = pd.read_csv('data.csv')

data.head()

,Instrument,Total Debt,Accounts Payable (CF),NAICS Industry Group Name,Inventories (CF),Delisted Quote Flag,Retained Earnings (Accumulated Deficit),Net Debt Incl. Pref.Stock & Min.Interest,Total Current Assets,Total Liabilities,...,Tangible Book Value Per Share,Net Sales,Operating Income,Instrument Is Active Flag,Tangible Book Value - Utility,"Total Assets, Reported",Depreciation And Amortization,Total Current Liabilities,Year,Bankrupt
0,910655,1866000.0,-235000.0,Motor Vehicle Body and Trailer Manufacturing,-9289000.0,1.0,49741000.0,-8580000.0,8.268900e+07,2.325000e+07,...,8.43993,3.603260e+08,40075000.0,0.0,8.712400e+07,1.177390e+08,413000.0,1.920900e+07,2000,1
1,1116521,10000000.0,-47000000.0,Communications Equipment Manufacturing,81000000.0,1.0,NaN,-184000000.0,3.043000e+09,2.422000e+09,...,5.59129,8.268000e+09,369000000.0,0.0,1.556000e+09,4.239000e+09,NaN,1.597000e+09,2000,1
2,895651,377000.0,NaN,Offices of Other Health Practitioners,289000.0,1.0,-18251000.0,-1483000.0,9.479000e+06,1.318600e+07,...,0.40670,5.497100e+07,-1056000.0,0.0,3.622000e+06,3.214500e+07,NaN,6.303000e+06,2000,1
3,1398702,NaN,NaN,Tobacco Manufacturing,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2000,1
4,1434621,NaN,NaN,Nondepository Credit Intermediation,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,2000,1


In [11]:
len(data)

345740

In [3]:
print('Unique CIKs:', len(data.Instrument.unique()))

Unique CIKs: 15221


In [4]:
data.columns

Index(['Instrument', 'Total Debt', 'Accounts Payable (CF)',
       'NAICS Industry Group Name', 'Inventories (CF)', 'Delisted Quote Flag',
       'Retained Earnings (Accumulated Deficit)',
       'Net Debt Incl. Pref.Stock & Min.Interest', 'Total Current Assets',
       'Total Liabilities', 'Company Common Name', 'CIK Number',
       'Total Equity', 'Tangible Book Value - Banks',
       'Tangible Book Value - Insurance',
       'Net Income/Starting Line, Cumulative',
       'Tangible Book Value - Reported', 'Cash and Short Term Investments',
       'Market Value for Company', 'Tangible Book Value Per Share',
       'Net Sales', 'Operating Income', 'Instrument Is Active Flag',
       'Tangible Book Value - Utility', 'Total Assets, Reported',
       'Depreciation And Amortization', 'Total Current Liabilities', 'Year',
       'Bankrupt'],
      dtype='object')

In [10]:
# replace all NAN values with 0 and non-NANs with 1
data2 = data.notnull().astype('int')
data2.Instrument, data2.Year, data2.Bankrupt = data.Instrument, data.Year, data.Bankrupt

data2['index'] = data.index

data2 = data2.groupby(['Instrument', 'Year', 'Bankrupt', 'index']).sum().reset_index()
data2['noNANs'] = data2.sum(axis=1) - data2['Instrument'] - data2['Year'] - data2['Bankrupt'] - data2['index']
data2.Instrument, data2.Year, data2.Bankrupt, data2.index = data.Instrument, data.Year, data.Bankrupt, data.index
data2 = data2[['index', 'Instrument','Year','Bankrupt', 'noNANs']].copy()
data2 = data2.sort_values(by=['Instrument', 'Year'])


data2

,index,Instrument,Year,Bankrupt,noNANs
25155,299306,20,2000,0,1
41232,219308,20,2001,0,25
57309,172609,20,2002,0,26
73386,237617,20,2003,0,25
89463,77763,20,2004,0,0
105540,31968,20,2005,0,17
121617,305487,20,2006,0,26
137694,255533,20,2007,0,26
153771,210915,20,2008,0,22
169848,163268,20,2009,0,24


In [ ]:
# first I need to remove the duplicate rows for Instrument-Year, based on the number of no-NANs values

# Secondly, There are Bankrupt companies recorded with label 0 (healthy)
# for years before backruptcy which somehow fucked everything up

# So you have to generate Bankrupt and healthy lists and also CIK-year dic
# again and check the results with the files in the backup folders

In [ ]:
# check CIK with duble-label
test = data[['Instrument','Bankrupt']].copy()
test2 = test.groupby(['Instrument']).count()

test3 = test2[test2.Bankrupt > 20]

test3.Bankrupt.value_counts()

In [ ]:
test2[test2.Bankrupt == 120]

In [8]:
data[(data.Instrument == 1525494) & (data.Year == 2019)]

,Instrument,Total Debt,Accounts Payable (CF),NAICS Industry Group Name,Inventories (CF),Delisted Quote Flag,Retained Earnings (Accumulated Deficit),Net Debt Incl. Pref.Stock & Min.Interest,Total Current Assets,Total Liabilities,...,Tangible Book Value Per Share,Net Sales,Operating Income,Instrument Is Active Flag,Tangible Book Value - Utility,"Total Assets, Reported",Depreciation And Amortization,Total Current Liabilities,Year,Bankrupt
333357,1525494,852427000.0,40503000.0,Other Information Services,NaN,0.0,-1.689683e+09,-2.152171e+09,6.381061e+09,8.673061e+09,...,8.19758,1.006878e+10,1.796465e+09,1.0,2.415542e+09,2.826651e+10,NaN,5.600948e+09,2019,0


## Healthy Companies Data

In [3]:
# extract the healthy dataframe
healthy_raw = data[data['Bankrupt'] == 0]

healthy_raw.head()

,Instrument,Total Debt,Accounts Payable (CF),NAICS Industry Group Name,Inventories (CF),Delisted Quote Flag,Retained Earnings (Accumulated Deficit),Net Debt Incl. Pref.Stock & Min.Interest,Total Current Assets,Total Liabilities,...,Tangible Book Value Per Share,Net Sales,Operating Income,Instrument Is Active Flag,Tangible Book Value - Utility,"Total Assets, Reported",Depreciation And Amortization,Total Current Liabilities,Year,Bankrupt
24200,1750,1.813590e+08,24659000.0,Aerospace Product and Parts Manufacturing,-46212000.0,0.0,1.845290e+08,1.731090e+08,5.081860e+08,4.005950e+08,...,10.44308,1.050608e+09,7.738100e+07,1.0,2.859420e+08,7.266300e+08,NaN,1.735860e+08,2000,0
24201,61478,4.990000e+07,45000000.0,Communications Equipment Manufacturing,-66700000.0,1.0,8.992000e+08,-4.846000e+08,1.376600e+09,4.625000e+08,...,14.01205,2.151800e+09,1.350000e+08,0.0,1.358300e+09,2.057800e+09,2.220000e+07,4.485000e+08,2000,0
24202,319126,6.402000e+06,-300000.0,"Navigational, Measuring, Electromedical, and C...",665000.0,0.0,-6.361000e+06,6.153000e+06,9.758000e+06,8.778000e+06,...,129.66559,2.937000e+07,-1.682000e+06,1.0,2.404000e+06,1.398600e+07,NaN,2.804000e+06,2000,0
24203,730469,4.830080e+08,NaN,Other Investment Pools and Funds,1255000.0,1.0,5.664900e+07,4.685940e+08,3.354840e+08,6.416570e+08,...,-1.79517,6.002820e+08,6.265100e+07,0.0,-4.843000e+07,9.089360e+08,NaN,1.704370e+08,2000,0
24204,6201,4.402000e+09,NaN,Scheduled Air Transportation,-36000000.0,0.0,4.733000e+09,2.867000e+09,3.969000e+09,1.475700e+10,...,33.90123,1.751600e+10,1.988000e+09,1.0,5.470000e+09,2.145500e+10,1.040000e+09,5.302000e+09,2000,0


In [4]:
for col in healthy_raw.columns:
    print(col)

print()
print(len(healthy_raw.columns))

Instrument
Total Debt
Accounts Payable (CF)
NAICS Industry Group Name
Inventories (CF)
Delisted Quote Flag
Retained Earnings (Accumulated Deficit)
Net Debt Incl. Pref.Stock & Min.Interest
Total Current Assets
Total Liabilities
Company Common Name
CIK Number
Total Equity
Tangible Book Value - Banks
Tangible Book Value - Insurance
Net Income/Starting Line, Cumulative
Tangible Book Value - Reported
Cash and Short Term Investments
Market Value for Company
Tangible Book Value Per Share
Net Sales
Operating Income
Instrument Is Active Flag
Tangible Book Value - Utility
Total Assets, Reported
Depreciation And Amortization
Total Current Liabilities
Year
Bankrupt

29


In [5]:
# drop non-quantitative columns
healthy = healthy_raw.drop(columns=['NAICS Industry Group Name', 'Delisted Quote Flag', 'Company Common Name',
                                    'CIK Number', 'Instrument Is Active Flag', 'Bankrupt'])

healthy.head()

,Instrument,Total Debt,Accounts Payable (CF),Inventories (CF),Retained Earnings (Accumulated Deficit),Net Debt Incl. Pref.Stock & Min.Interest,Total Current Assets,Total Liabilities,Total Equity,Tangible Book Value - Banks,...,Cash and Short Term Investments,Market Value for Company,Tangible Book Value Per Share,Net Sales,Operating Income,Tangible Book Value - Utility,"Total Assets, Reported",Depreciation And Amortization,Total Current Liabilities,Year
24200,1750,1.813590e+08,24659000.0,-46212000.0,1.845290e+08,1.731090e+08,5.081860e+08,4.005950e+08,3.260350e+08,2.859420e+08,...,8.250000e+06,4.812300e+08,10.44308,1.050608e+09,7.738100e+07,2.859420e+08,7.266300e+08,NaN,1.735860e+08,2000
24201,61478,4.990000e+07,45000000.0,-66700000.0,8.992000e+08,-4.846000e+08,1.376600e+09,4.625000e+08,1.595300e+09,1.358300e+09,...,5.345000e+08,1.097519e+10,14.01205,2.151800e+09,1.350000e+08,1.358300e+09,2.057800e+09,2.220000e+07,4.485000e+08,2000
24202,319126,6.402000e+06,-300000.0,665000.0,-6.361000e+06,6.153000e+06,9.758000e+06,8.778000e+06,5.208000e+06,2.404000e+06,...,2.490000e+05,3.480000e+06,129.66559,2.937000e+07,-1.682000e+06,2.404000e+06,1.398600e+07,NaN,2.804000e+06,2000
24203,730469,4.830080e+08,NaN,1255000.0,5.664900e+07,4.685940e+08,3.354840e+08,6.416570e+08,2.672790e+08,-4.843000e+07,...,1.441400e+07,NaN,-1.79517,6.002820e+08,6.265100e+07,-4.843000e+07,9.089360e+08,NaN,1.704370e+08,2000
24204,6201,4.402000e+09,NaN,-36000000.0,4.733000e+09,2.867000e+09,3.969000e+09,1.475700e+10,6.698000e+09,5.470000e+09,...,1.535000e+09,NaN,33.90123,1.751600e+10,1.988000e+09,5.470000e+09,2.145500e+10,1.040000e+09,5.302000e+09,2000


In [6]:
# replace all NAN values with 0 and non-NANs with 1
healthy = healthy.notnull().astype('int')

# except for the instruments and the year
healthy['Instrument'] = healthy_raw['Instrument']
healthy['Year'] = healthy_raw['Year']

print(healthy.shape)

healthy.head()

(321540, 23)


,Instrument,Total Debt,Accounts Payable (CF),Inventories (CF),Retained Earnings (Accumulated Deficit),Net Debt Incl. Pref.Stock & Min.Interest,Total Current Assets,Total Liabilities,Total Equity,Tangible Book Value - Banks,...,Cash and Short Term Investments,Market Value for Company,Tangible Book Value Per Share,Net Sales,Operating Income,Tangible Book Value - Utility,"Total Assets, Reported",Depreciation And Amortization,Total Current Liabilities,Year
24200,1750,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,2000
24201,61478,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,2000
24202,319126,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,1,2000
24203,730469,1,0,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,0,1,2000
24204,6201,1,0,1,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,2000


In [7]:
for col in healthy.columns:
    print(col)

print()
print(len(healthy.columns))

Instrument
Total Debt
Accounts Payable (CF)
Inventories (CF)
Retained Earnings (Accumulated Deficit)
Net Debt Incl. Pref.Stock & Min.Interest
Total Current Assets
Total Liabilities
Total Equity
Tangible Book Value - Banks
Tangible Book Value - Insurance
Net Income/Starting Line, Cumulative
Tangible Book Value - Reported
Cash and Short Term Investments
Market Value for Company
Tangible Book Value Per Share
Net Sales
Operating Income
Tangible Book Value - Utility
Total Assets, Reported
Depreciation And Amortization
Total Current Liabilities
Year

23


In [8]:
healthy = healthy.groupby(['Instrument', 'Year']).sum().reset_index()
healthy['noNANs'] = healthy.sum(axis=1) - healthy['Instrument'] - healthy['Year']
healthy = healthy[['Instrument', 'Year', 'noNANs']]

healthy.head()

,Instrument,Year,noNANs
0,20,2000,19
1,20,2001,19
2,20,2002,19
3,20,2003,19
4,20,2004,19


In [9]:
healthy.noNANs.max()

21

## Bankrupt Companies Dates Data

In [10]:
# read the bankruptcy dates
bankrupt = pd.read_csv('bankruptcy_dates.csv', index_col=0)
# sort by date
bankrupt.sort_values(by=['Date'], inplace=True)

bankrupt

,CIK,Company,Date
8,6125,Keystone Camera Products,1992
29,797422,Hi-Def Enterprise Inc,1993
23,822373,Namsco Corp,1994
12,87817,Scientific Radio Systems Inc,1995
645,1129981,OGLEBAY NORTON CO /OHIO/,2004
88,225463,HUFFY CORP,2004
519,1069201,CROSS MEDIA MARKETING CORP,2004
122,701345,US AIRWAYS GROUP INC,2004
598,1107080,ENVIRO ENERGY CORP,2004
188,801550,DVI INC,2004


In [11]:
# keep only bankrupt companies that went bankrupt after 2006
bankrupt = bankrupt[bankrupt.Date > 2005]
dates = bankrupt.groupby(['Date']).count()

dates

,CIK,Company
Date,,
2006,59,59
2007,85,85
2008,94,94
2009,177,177
2010,93,93
2011,66,66
2012,72,72
2013,64,64
2014,56,56


# The Situation

So here we have 14 years, in each a specific number of bankrupt comapnies. Now we want to extract 5 healthy companies for each bankrupt one. Plus, this data should be collected from a period of year-7 to year-5.

So for example, we need data for 93 * 5 healthy companies in years 2004 to 2008 (inclusive), for 93 companies that went bankrupt in 2010.


On the other hand, we have a dataframe of healthy companies CIK code for each year (2000-2019) and a column showing the number of non-NAN values for each row. A no-NANs equal to 21 indicate that the company for that year has full data.
Now our goal is to pick those 5 company for each bankrupt companies from ones with the highest no-NANs values.



## The Strategy

For each time period, we calculate the sum of no-NANs for that period and pick the n maximums; n being the number of companiets that went bankrupt in year+1. So we have to do it 14 times (2007-2019).

In [ ]:
healthy.head()

In [ ]:
# write a function for the task

def extract_healthy(df, year, n):
    '''
    The function takes the year in which n companies went bankrupt,
    and from the healthy dataframe, returns a CIK lists of 5 * n
    healthy companies which 6 years to 2 years before that year has
    the fullest dataset.
    '''
    
    # first remove all the extra rows for years not in the period
    df_temp = df[(df.Year < year-1) & (df.Year > year-7)]
    # group the dataframe by instrument, sum over the no-NANs
    df_temp = df_temp.groupby(['Instrument']).sum()
    # drop the now useless year column
    df_temp.drop(columns=['Year'], inplace=True)
    # descending-sort the companies based on noNANs magnitude
    df_temp.sort_values(by=['noNANs'], ascending=False, inplace=True)
    # cut the first n * 5 ones
    df_temp = df_temp[:5*n]
    # make a list of their CIK cides
    CIK_list = df_temp.index.to_list()
    # drop the healthy companies already picked from the df
     
    
    return CIK_list
    

In [ ]:
# keep the original df safe, just in case
healthy_copy = healthy.copy()

# create a dic storing the CIK lists for different years
CIK_year = {}

# loop over all available years in our dataset
for year in range(2019, 2005, -1):
    # take the number of companies went bankrupt in that year
    n = int(dates[dates.index == year]['Company'])
    # take the CIK list
    CIK_list = extract_healthy(healthy_copy, year, n)
    # add the list to the dictionary
    CIK_year[year] = CIK_list
    

print(CIK_year)

In [ ]:
# write the dictionary
with open('CIK_year.txt', 'wb') as f:
    pickle.dump(CIK_year, f)

## Second Strategy

This time I will create a dataframe, with the rows being healthy companies, and the columns being the 5-year periods, so to calculate the fullness of each healthy company for each period for an optimal selection.

In [12]:
healthy.head()

,Instrument,Year,noNANs
0,20,2000,19
1,20,2001,19
2,20,2002,19
3,20,2003,19
4,20,2004,19


In [13]:
def score_cal(df, year):
        '''
        The function takes the healthy df and a year, and calculate the
        healthy companies' scores (#non-NAN-values) for 6 years to 2
        years before that year, and returns it in a dataseries.
        '''
        
        # first remove all the extra rows for years not in the period
        df_temp = df[(df.Year < year-1) & (df.Year > year-7)]
        # group the dataframe by instrument, sum over the no-NANs
        df_temp = df_temp.groupby(['Instrument']).sum()

        return df_temp.noNANs


    
def extract_healthy_2(df):
    '''
    This function takes a healthy dataframe and and returns a
    dataframe stating each company's score (non-NAN values) for
    a period.
    '''

    # create a new dataframe to store each company score per period
    df_out = df.groupby(['Instrument']).sum()
    
    for year in range(2019, 2005, -1):
        # set the column title
        title = str(year) + ':(' + str(year-6) + '-' + str(year-2) + ')'
        df_out[title] = np.around((score_cal(df, year) / 105 * 100), decimals=2)
        
    df_out.drop(columns=['Year', 'noNANs'], inplace=True)
        
    return df_out


In [14]:
dfout = extract_healthy_2(healthy)

dfout

,2019:(2013-2017),2018:(2012-2016),2017:(2011-2015),2016:(2010-2014),2015:(2009-2013),2014:(2008-2012),2013:(2007-2011),2012:(2006-2010),2011:(2005-2009),2010:(2004-2008),2009:(2003-2007),2008:(2002-2006),2007:(2001-2005),2006:(2000-2004)
Instrument,,,,,,,,,,,,,,
20,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48
1750,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24
1800,95.24,94.29,93.33,92.38,91.43,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48
1923,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,91.43,92.38,93.33
1961,90.48,90.48,89.52,88.57,87.62,86.67,85.71,85.71,85.71,85.71,86.67,74.29,75.24,76.19
2034,96.19,96.19,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24
2062,61.90,62.86,63.81,64.76,65.71,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67,66.67
2098,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24
2135,85.71,85.71,85.71,85.71,85.71,85.71,85.71,85.71,85.71,86.67,87.62,88.57,89.52,90.48


## Final Decision

In [15]:
# for the first step, remove the CIK with no filed/recorded 10-K forms
# read the csv file
cik_no10K = pd.read_csv('cik_without_10k.csv', header=None)

bad_cik = cik_no10K[0].to_list()

cik_no10K.head()

,0
0,1423221
1,704440
2,1616262
3,895564
4,315126


In [16]:
# just check a list of full rows companies, plus one totally empty one
check_list = [98840, 98222, 1001082, 1000697, 1000229, 1000180]

for cik in check_list:
    if cik in bad_cik:
        print(cik, 'is a bad CIK!')

print(len(bad_cik), 'bad CIK codes!')
healthy_fullness = dfout.reset_index()
healthy_fullness.head()

933 bad CIK codes!


,Instrument,2019:(2013-2017),2018:(2012-2016),2017:(2011-2015),2016:(2010-2014),2015:(2009-2013),2014:(2008-2012),2013:(2007-2011),2012:(2006-2010),2011:(2005-2009),2010:(2004-2008),2009:(2003-2007),2008:(2002-2006),2007:(2001-2005),2006:(2000-2004)
0,20,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48
1,1750,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24
2,1800,95.24,94.29,93.33,92.38,91.43,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48
3,1923,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,91.43,92.38,93.33
4,1961,90.48,90.48,89.52,88.57,87.62,86.67,85.71,85.71,85.71,85.71,86.67,74.29,75.24,76.19


In [17]:
# check the CIK formats in dataframe and dictionary
# to make sure they're comparable
print(type(bad_cik[1]))
print(type(healthy_fullness.Instrument[1]))


<class 'int'>
<class 'numpy.int64'>


In [22]:
# calculate the number of unique CIK in the healthy list
len(healthy_fullness.Instrument.unique())

# drop companies with no filed 10-K form
healthy = healthy_fullness[~healthy_fullness.Instrument.isin(bad_cik)]
# check to make sure everything is okay and under control
print(len(healthy_fullness), '-', len(set(bad_cik)), '=', len(healthy))

healthy.head()

14761 - 292 = 14469


,Instrument,2019:(2013-2017),2018:(2012-2016),2017:(2011-2015),2016:(2010-2014),2015:(2009-2013),2014:(2008-2012),2013:(2007-2011),2012:(2006-2010),2011:(2005-2009),2010:(2004-2008),2009:(2003-2007),2008:(2002-2006),2007:(2001-2005),2006:(2000-2004)
0,20,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48
1,1750,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24,95.24
2,1800,95.24,94.29,93.33,92.38,91.43,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48
3,1923,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,90.48,91.43,92.38,93.33
4,1961,90.48,90.48,89.52,88.57,87.62,86.67,85.71,85.71,85.71,85.71,86.67,74.29,75.24,76.19


In [36]:
[23, 44] + [444, 88, 'p']

[23, 44, 444, 88, 'p']

In [38]:
## temporary
# ----------------------------------------------------------------
urgent_h = {}
urgent_b = {}

for column in healthy:
    if column == 'Instrument':
        pass
    else:
        year = int(column[:4])
        n = dates.Company[year]
        healthy.sort_values(by=[column], ascending=False, inplace=True)
        CIK_list = healthy.Instrument.to_list()
        urgent_h[column] = CIK_list[:5*n]
        urgent_b[column] = bankrupt[bankrupt.Date == year].CIK.to_list()

with open('urgent_h.txt', 'wb') as f:
    pickle.dump(urgent_h, f)
    
with open('urgent_b.txt', 'wb') as f:
    pickle.dump(urgent_b, f)
    
# ----------------------------------------------------------------


/Users/user/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


In [19]:
# create a new dictionary to save the fullest
# healthy companies for each period
cik_codes = {2000:[], 2001:[], 2002:[], 2003:[], 2004:[],
             2005:[], 2006:[], 2007:[], 2008:[], 2009:[],
             2010:[], 2011:[], 2012:[], 2013:[], 2014:[],
             2015:[], 2016:[], 2017:[]}

def CIK_picking(df, year, n):
    '''
    this function takes the healthy fullness dataframe,
    a year, and the number of bankrupt companies in the year (n),
    and produces a list including 5n healthy companies with
    the fullest rows in the period 7 to 2 years before that year,
    plus CIK of bankrupt companies in that year.
    The function, then, adds these CIK codes to the cik_codes'
    keys for each year in the period of 7-2 years before that year.
    And finally write the dictionary in a pickle file.
    '''
    
    # descending-sort the companies for that year
    col = str(year) + ':(' + str(year-6) + '-' + str(year-2) + ')'
    df.sort_values(by=[col], ascending=False, inplace=True)
    # make a list of their CIK cides
    CIK_list = df.Instrument.to_list()
    # cut the first n * 5 ones
    max_list = CIK_list[:5*n]
    # add bankrupt CIK in the 'year' to the list
    final_list = max_list + bankrupt[bankrupt.Date == year].CIK.to_list()
    
    for yr in range(year-6, year-1):
        cik_codes[yr] += final_list


# run the healthy_picking function for 14 years (2006-19)
for year in range(2006, 2020):
    # take the number of companies went bankrupt in that year
    n = dates.Company[year]
    # add the CIK lists to the cik_codes dictionary
    CIK_picking(healthy, year, n)

/Users/user/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [ ]:
# Clear each cik_codes dictionary's key's list to make sure
# there ain't repetitive elements (CIKs) in them

# create a list, to store all CIKs
all_CIKs = []

for year in range(2000, 2018):
    cik_codes[year] = list(set(cik_codes[year]))
    all_CIKs += cik_codes[year]
    
# write the dictionary
with open('CIK_year.txt', 'wb') as f:
    pickle.dump(cik_codes, f)
    

# remove repetitive CIKs
all_CIKs = list(set(all_CIKs))
# the number of CIKs - the total number of bankrupt CIKs
# to check the number of healthy CIKs in our data
print('Final number of healthy companies in our dataset:', len(all_CIKs) - dates.Company.sum())
print('And the number of bankrupt companies in our dataset:', dates.Company.sum())